In [1]:
import polars as pl
import re

# 学習データからテキストのみをリストで抽出
FILE_PATH = "../第6章/news+aggregator/train.txt"
df = pl.read_csv(FILE_PATH, separator="\t", new_columns=["text", "categoory"])
text_list = df["text"].to_list()

#　記号を取り除く
cleaned_text_list = [re.sub(r'[^a-zA-Z\s]', "", text) for text in text_list]
word_list = [text.split() for text in cleaned_text_list]
word_list
#　記号を取り除く
word_frequency_dict = {}
# 単語リストをループして出現頻度をカウント
for words in word_list:
    for word in words:
        if word in word_frequency_dict:
            word_frequency_dict[word] += 1
        else:
            word_frequency_dict[word] = 1

sorted_word_frequency_dict = dict(sorted(word_frequency_dict.items(), key=lambda item: item[1], reverse=True)) #降順にソート


word_id_map = {}    #単語とidの辞書型
id = 1
for key, value in sorted_word_frequency_dict.items():
    if value == 1: #出現頻度が1のidは0
        word_id_map[key] = 0
    else:
        word_id_map[key] = id
        id += 1
# wordのidは0~9509の計9510個のidが存在する→つまりone-hotベクトルは9510次元
onehot_dim = max(word_id_map.values()) + 1

In [8]:
import torch
from torch import nn

class LSTMModel(nn.Module):
    def __init__(self,onehot_dim=onehot_dim ,embedding_dim = 300, hidden_dim = 50, output_dim = 4):
        super().__init__()
        self.emb = nn.Embedding(onehot_dim, embedding_dim)
        self.LSTM = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim, bias=True)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x, h=None):
        x = self.emb(x)
        y, h= self.LSTM(x, h)
        y = y[-1]
        y = self.linear(y)
        y = self.softmax(y)
        return y

In [9]:
def get_index_vector(words, word_id_map):
    # まず、必要な形状のテンソルを初期化
    vectors = torch.zeros(len(words))
    # 各単語の位置に1をセット
    for i, word in enumerate(words):
        vectors[i] = word_id_map[word]
    return vectors.long()


In [10]:
model = LSTMModel()
data = get_index_vector(word_list[0],word_id_map)
model.forward(data)

tensor([0.2601, 0.2715, 0.2210, 0.2474], grad_fn=<SoftmaxBackward0>)